In [2]:
import pandas as pd
# import  modin.pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
from gensim.models import word2vec
path = '/home/kesci/input/bytedance/first-round/'
import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency
import time
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics.pairwise import cosine_similarity
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D, concatenate
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
#tfidf
from sklearn.feature_extraction.text import TfidfVectorizer,TfidfTransformer
from scipy.sparse import coo_matrix, vstack,hstack
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import KeyedVectors
import spacy  # For preprocessing
import multiprocessing
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
import gc 
from gensim.models.phrases import Phrases, Phraser
from gensim import corpora
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from gensim import corpora
import Levenshtein
#线性代数中的范数
from scipy.linalg import norm
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.datasets import  make_classification
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV,Ridge, Lasso
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.metrics.pairwise import euclidean_distances,manhattan_distances,cosine_distances
!pip install  --index "http://pypi/simple" --trusted-host pypi  fuzzywuzzy
from fuzzywuzzy import fuzz
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
!ls /home/kesci/input/bytedance/first-round/

test.csv  train.csv


In [3]:
#读数据
train = pd.read_csv('lgb_train.csv'
    )
# train = tra_data.get_chunk(5000000)

test = pd.read_csv('lgb_test.csv'
    )

In [5]:
#Build doc

def train_word2vec(init = False):
    global test,train
    if init == True:
        title = [s.split() for s in train['title'].values]
        query = [s.split() for s in train['query'].values]
        doc = title + query
        title = [s.split() for s in test['title'].values]
        query = [s.split() for s in test['query'].values]
        doc += title
        doc += query
        gc.collect()
        # del test,title,query
        #build vocabulary
        import gensim
        model = gensim.models.Word2Vec(size=128, window=3, min_count=1, sg=1, workers=10)
        model.build_vocab(doc)  # prepare the model vocabulary
        model.train(sentences=doc, total_examples=model.corpus_count, epochs=model.iter)
        model.save('word2vec.model')
        #model = Word2Vec.load("word2vec.model")
        del doc
train_word2vec(True)

Feature Engineering

In [4]:
def Leven_distance_list(row): 
    '''
    #编辑距离 脱敏后的词语
    '''
    str1 = row['query'].split()
    str2 = row['title'].split()
    dp=np.zeros((len(str1)+1,len(str2)+1))
    m=len(str1)
    n=len(str2)
    for k in range(1,m+1):
         dp[k][0]=k
    for k in range(1,n+1):
         dp[0][k]=k
    for k in range(1,m+1):
         for j in range(1,n+1):
            dp[k][j]=min(dp[k-1][j],dp[k][j-1])+1 #这里表示上边和下边的数值最小数值
            if str1[k-1]==str2[j-1]:
                dp[k][j]=min(dp[k][j],dp[k-1][j-1])
            else:
                dp[k][j]=min(dp[k][j],dp[k-1][j-1]+1)
    return int(dp[-1][-1])
def seqratio(row):
    '''
    基于编辑距离的相似度
    '''
    edit_distance = Leven_distance_list(row)
    len1 = len(row['query'].split())
    len2 = len(row['title'].split())
    return (len1 + len2 - edit_distance) / (len1 + len2)
#train.head().apply(seqratio,axis=1)
def jaccard_similarity(row):
    '''
    基于杰卡迪的相似度
    '''
    str1 = row['query']
    str2 = row['title']
    cv = CountVectorizer(tokenizer= lambda s : s.split())
    corpus = [str1,str2]
    vec = cv.fit_transform(corpus).toarray()
    inter = np.sum(np.min(vec,axis=0))
    merge = np.sum(np.max(vec,axis=0))
    return 1.0* inter/merge

def tf_similarity(row):
    '''
    基于tf的余弦相似度特征
    '''
    s1 = row['title']
    s2 = row['query']
    cv = CountVectorizer(tokenizer=lambda x : x.split())
    corpus = [s1,s2]
    vector = cv.fit_transform(corpus).toarray()
    return np.dot(vector[0],vector[1]) / (norm(vector[0])*norm(vector[1]))
#     return cosine_similarity(vector)[1][0]

def tfidf_similarity(row):
    '''
    基于tf-idf的余弦相似度特征
    '''
    s1 = row['title']
    s2 = row['query']
    tfidf = TfidfVectorizer(tokenizer=lambda x :x.split())
    corpus = [s1,s2]
    vector = tfidf.fit_transform(corpus).toarray()
    return np.dot(vector[0],vector[1]) / (norm(vector[0])*norm(vector[1]))
def lda(df):
     '''
     主题模型
     '''
     query = df['query'].values
     query = list(map(lambda x : x.split(' '),query))
     dictionary = corpora.Dictionary(query)
     corpus = [dictionary.doc2bow(words) for words in query]
     lda = LdaModel(corpus=corpus, id2word=dictionary, num_topics=50)
     vec = lda.inference(corpus)
     res = np.argmax(vec[0],axis=1)
     df['lda'] = res.reshape(-1,1)
     return df
def Levenshtein_dis_rate(item):
    '''
    计算prefix和title的距离比例,即距离/长度
    '''
    str1=item['query'].split(' ')
    str2=item['title'].split(' ')
    leven_distance=item['distance']
    length=max(len(str1),len(str2))
    return leven_distance/length   
    
def LCS(item):
    '''
    计算str1和str2之间的最长公共子串的长度
    '''
    str1=item['query'].split(' ')
    str2=item['title'].split(' ')
    if len(str1)==0 or len(str2) ==0:
         return 0
    len1 = len(str1)
    len2 = len(str2)

    c = np.zeros((len2+1,),dtype=np.int32)
    max_len = 0
    for i in range(len1):
        for j in range(len2-1,-1,-1):
            if str1[i] == str2[j]:
                c[j+1] = c[j] + 1
                if c[j+1]>=max_len:
                    max_len = c[j+1]
            else:
                c[j+1] = 0
    return max_len
def rp(df):
    '''
    recall precision acc
    '''
    def rp_query_in_title(item): #计算acc，recall，precisoin
        query=item['query'].split(' ')
        title=item['title'].split(' ')
        len_title=len(title)
        len_query=len(query)
        len_comm_xx=len(set(query)&set(title))
        recall=len_comm_xx/(len_query+0.01)
        precision=len_comm_xx/(len_title+0.01)
        acc=len_comm_xx/(len_title+len_query-len_comm_xx+0.01)
        return [recall,precision,acc]
    
    a = df.apply(rp_query_in_title,axis=1)
    a = np.array(a.values.tolist())
    # print(a.shape)
    df['recall'] = a[:,0]
    df['precision'] = a[:,1]
    df['acc'] = a[:,2]
    return df  
def w2v_simi(df):
    '''
    word2vec相似度特征
    '''
    w2v = Word2Vec.load("word2vec.model")
    def w2v_similarity(row):
        s1 = row['query']
        s2 = row['title']
        def sentence_vector(s):
            words = s.split()
            v = np.zeros(128)
            for i in words:
                v+=w2v[i]
            v/=len(s)
            return v
        v1,v2 = sentence_vector(s1),sentence_vector(s2)
        return np.dot(v1,v2) / (norm(v1)*norm(v2))
    return df.apply(w2v_similarity,axis=1)

def query_complete_in_title(item):
    '''
    是否都在title里
    '''
    query=item['query'].split(' ')
    title=item['title'].split(' ')
    n=0
    for word in query:
        if word in title:
            n+=1
    return 1 if n==len(query) else 0
    
def title_complete_in_query(item):
    '''
    是否都在query里
    '''
    query=item['query'].split(' ')
    title=item['title'].split(' ')
    n=0
    for word in title:
        if word in query:
            n+=1
    return 1 if n==len(title) else 0
def query_and_title(item):
    '''
    query 与 title 交集
    '''
    query=item['query'].split(' ')
    title=item['title'].split(' ')  
    num=0
    for word in query:
        if word in title:
            num+=1
    return num
def query_in_title_of_query(item):
    '''
    query 与 title交集/query
    '''
    query=item['query'].split(' ')
    title=item['title'].split(' ')  
    num=0
    if title!=' ':
        for word in query:
            if word in title:
                num+=1
        return num/len(query)
    return 0
def query_in_title_of_title(item):
    '''
    query 与 title交集/ title
    '''
    query=item['query'].split(' ')
    title=item['title'].split(' ')  
    num=0
    if title!=' ':
        for word in query:
            if word in title:
                num+=1
        return num/len(title)
    return 0
def fuzz_ratio(row):
    '''
    Simple Ratio
    '''
    s1 = row['query']
    s2 = row['title']
    return fuzz.ratio(s1,s2)
    
def partial_ratio(row):
    '''
    Partial  Ratio
    '''
    s1 = row['query']
    s2 = row['title']
    return fuzz.partial_ratio(s1,s2)
    
def token_sort_ratio(row):
    '''
    token_sort_ratio
    '''
    s1 = row['query']
    s2 = row['title']
    return fuzz.token_sort_ratio(s1,s2)
    
def token_set_ratio(row):
    '''
    token_set_ratio
    '''
    s1 = row['query']
    s2 = row['title']
    return fuzz.token_set_ratio(s1,s2)
    
def Word_Share(df):
    '''
    Word_Share
    '''
    def get_weight(count, eps=10000, min_count=2):
        return 0 if count < min_count else 1 / (count + eps)

    # title = [s.split() for s in df['title'].values]
    # query = [s.split() for s in df['query'].values]
    # words = title + query
    train_qs = pd.Series(df['query'].tolist() + df['title'].tolist()).astype(str)
    words = (" ".join(df)).lower().split()
    counts = Counter(words)
    weights = {word: get_weight(count) for word, count in counts.items()}
    
    # stops = set(stopwords.words("english"))
    def word_shares(row):
    	q1words = set(str(row['query']).split())
    	q2words = set(str(row['title']).split())
    
    	shared_words = q1words.intersection(q2words)
    	shared_weights = [weights.get(w, 0) for w in shared_words]
    # 	total_weights = [weights.get(w, 0) for w in q1words] + [weights.get(w, 0) for w in q2words]
    	
    # 	R1 = np.sum(shared_weights) / (np.sum(total_weights)+10000) #tfidf share
    # 	print(f'np.sum(shared_weights) = {np.sum(shared_weights)}, total_weights = {total_weights}')
    	R2 = len(shared_words) / (len(q1words) + len(q2words)) #count share
    	return '{}:{}'.format( R2, len(shared_words))
        
        # Concat train and test datasets
    # df = pd.concat([train, test])
    df['word_shares'] = df.apply(word_shares, axis=1, raw=True)
    df['tfidf_word_match'] = df['word_shares'].apply(lambda x: float(x.split(':')[0]))
    df['shared_count']     = df['word_shares'].apply(lambda x: float(x.split(':')[1]))
    return df.drop('word_shares',axis=1)

def num_of_query_each_title(df):
    '''
    query_id对应的个数
    '''
    num_list=[]
    query_dict={}
    for i in tqdm(range(len(df))):
        if df.iloc[i]['query_id'] not in query_dict:
            num=len(df[df['query_id']==df.iloc[i]['query_id']])
            query_dict[df.iloc[i]['query_id']]=num
            num_list.append(num)
        else:
            num_list.append(query_dict[df.iloc[i]['query_id']])
    return num_list
def Title_of_query(df):
    '''
    Title_of_query
    '''
    def title_of_query(df):
        start=time.time()
        result=[]
        for i in df.groupby(['query_id'])['query_id']:
            temp=df[df['query_id']==i[0]]['title'].values
            temp=list(map(lambda x:x.split(' '),temp))
            max_len=0
            sum_len=0
            min_len=99999
            for line in temp:
                l=len(line)
                max_len=max(max_len,l)
                min_len=min(min_len,l)
                sum_len+=l
            average_len=sum_len/len(temp)
            for j in range(len(i[1])):
                result.append([max_len,min_len,average_len])
        print('用时:',time.time()-start)
        return result
    a = title_of_query(df)
    a = np.array(a)
    # print(a.shape)
    df['max_len'] = a[:,0]
    df['min_len'] = a[:,1]
    df['average_len'] = a[:,2]
    return df
  
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics.pairwise import haversine_distances
def Euclidean_distances(df):
    '''
    euclidean_distances
    '''
    w2v = Word2Vec.load("word2vec.model")
    def sentence_vector(row):
        s1 = row['query']
        s2 = row['title']
        def sentence_vector(s):
            words = s.split()
            v = np.zeros(128)
            for i in words:
                v+=w2v[i]
            v/=len(s)
            return v
        v1,v2 = sentence_vector(s1),sentence_vector(s2)
        return  euclidean_distances([v1],[v2])[0][0]
    return df.apply(sentence_vector,axis=1)


def Manhattan_distances(df):
    '''
    euclidean_distances
    '''
    w2v = Word2Vec.load("word2vec.model")
    def sentence_vector(row):
        s1 = row['query']
        s2 = row['title']
        def sentence_vector(s):
            words = s.split()
            v = np.zeros(128)
            for i in words:
                v+=w2v[i]
            v/=len(s)
            return v
        v1,v2 = sentence_vector(s1),sentence_vector(s2)
        return manhattan_distances([v1],[v2])[0][0]
    return df.apply(sentence_vector,axis=1)


def Cosine_distances(df):
    '''
    euclidean_distances
    '''
    w2v = Word2Vec.load("word2vec.model")
    def sentence_vector(row):
        s1 = row['query']
        s2 = row['title']
        def sentence_vector(s):
            words = s.split()
            v = np.zeros(128)
            for i in words:
                v+=w2v[i]
            v/=len(s)
            return v
        v1,v2 = sentence_vector(s1),sentence_vector(s2)
        return cosine_distances([v1],[v2])[0][0]
    return df.apply(sentence_vector,axis=1)



#w2v d2v相似度 其他..


In [5]:
#Basic statistical features sometimes make sense in model accuracy. We can call them meta features.
def feature_extraction(data):
    
    #统计特征
    
    #1. Number of query words 
    data['query_num'] = data['query'].apply(lambda x : len(str(x).split()))
    #2. Number of title words 
    data['title_num'] = data['title'].apply(lambda x : len(str(x).split()))
    #3. diffence between query num and title num
    data['diff_len_word'] = np.abs(data['query_num'] - data['title_num'])
    #4. query length
    data['len_q1'] = data['query'].apply(lambda x: len(str(x)))
    #5. title length
    data['len_q2'] = data['title'].apply(lambda x: len(str(x)))
    #6. diffence between query length and title
    data['diff_len'] = np.abs(data['len_q2'] - data['len_q1'])
    #7. query char length
    data['len_char_q1'] = data['query'].apply(lambda x: len(str(x).replace(' ', '')))
    #8. title char length
    data['len_char_q2'] = data['title'].apply(lambda x: len(str(x).replace(' ', '')))
    #9. diffence between query char length and title char length
    data['diff_len_char'] = np.abs(data['len_char_q1'] - data['len_char_q2'])
    #10. avg_world_query
    data['avg_world_len1'] = data['len_char_q1'] / data['query_num']
    #11. avg_world_title
    data['avg_world_len2'] = data['len_char_q2'] / data['title_num']
    #12. diffence between avg_world_query and avg_world_title
    data['diff_avg_word'] = np.abs(data['avg_world_len1'] - data['avg_world_len2'])
    #13. common_words
    data['common_words'] = data.apply(lambda x: len(set(str(x['query']).split()).intersection(set(str(x['title']).split()))),axis=1)
    ##14. Number of query unique words in the text ##
    data['query_unique_num'] = data['query'].apply(lambda x : len(set(str(x).split())))
    ##15. Number of title unique words in the text ##
    data['title_unique_num'] = data['title'].apply(lambda x : len(set(str(x).split())))
    ##16. Average length of the query words in the text ##
    data['mean_query_len'] = data['query'].apply(lambda x : np.mean([len(w) for w in str(x).split()]))
    ##17. Average length of the title words in the text ##
    data['mean_title_len'] = data['title'].apply(lambda x : np.mean([len(w) for w in str(x).split()]))
    #18. 19. 21. Title_of_query  max min avg
    # data = Title_of_query(data)
    #21. num_of_query_each_title
    # data['num_of_query_each_title'] = num_of_query_each_title(data)
    ## 相似度特征
    
    #18. Edit distance
    data['distance'] = data.apply(Leven_distance_list,axis=1)
    #19. edit distance ratio
    data['seqratio'] = data.apply(seqratio,axis=1)
    #20. jaccard_similarity
    data['jaccard_similarity'] = data.apply(jaccard_similarity,axis=1)
    #21. tfidf_similarity
    data['tfidf_similarity'] = data.apply(tfidf_similarity,axis=1)
    #22. word2vec similarity
    data['w2v_simi'] = w2v_simi(data)
    #23. tf_similarity
    data['tf_similarity'] = data.apply(tf_similarity,axis=1)
    #24.Simple Ratio
    data['fuzz_ratio'] = data.apply(fuzz_ratio,axis=1)
    #25. Partial Ratio
    data['partial_ratio'] = data.apply(partial_ratio,axis=1)
    #27. Token Sort Ratio
    data['token_sort_ratio'] = data.apply(token_sort_ratio,axis=1)
    #28. Token Set Ratio
    data['token_set_ratio'] = data.apply(token_set_ratio,axis=1)
    
    #其他特征
    
    #29. Levenshtein_dis_rate
    data['Levenshtein_dis_rate'] = data.apply(Levenshtein_dis_rate,axis=1)
    #30. 最长上升子序列
    data['LCS'] = data.apply(LCS,axis=1)
    #31/ LDA 提取主题后，找到最大主题词的id
    data = lda(data)
    #32. 是否都在title里面
    data['query_complete_in_title'] = data.apply(query_complete_in_title,axis=1)
    #33. 是否都在query里面
    data['title_complete_in_query'] = data.apply(title_complete_in_query,axis=1)
    #34. query 与 title 交集
    data['query_and_title'] = data.apply(query_and_title,axis=1)
    #35. query 与 title交集/query
    data['query_in_title_of_query'] = data.apply(query_in_title_of_query,axis=1)
    #36. query 与 title交集/title
    data['query_in_title_of_title'] = data.apply(query_in_title_of_title,axis=1)
    #37. recall
    #38. precision
    #39. acc
    data = rp(data)
    #40. tfidf_word_match
    #41. shared_count
    data = Word_Share(data)
    #42. num_of_query_each_title
    # data['num_of_query_each_title'] = num_of_query_each_title(data)
    #43.max
    #44.min
    #45.avg
    data = Title_of_query(data)
    #46.euclidean_distances
    data['euclidean_distances'] = Euclidean_distances(data)
    #47.manhattan_distances
    data['manhattan_distances'] = Manhattan_distances(data)
    #48.cosine_distances
    data['cosine_distances'] = Cosine_distances(train)
    print(data.head()) 
    
    return data
train = feature_extraction(train)
#保存处理好的文件
train.to_csv('lgb_train.csv',index=False)
test = feature_extraction(test)
test.to_csv('lgb_test.csv',index=False)

In [5]:
#划分的用于普通lgb，train用于交叉验证lgb
#X_train,X_test, y_train, y_test = train_test_split(train,train['label'],test_size=0.3, random_state=0)
X_train,X_test, y_train, y_test = train_test_split(train.drop('label',axis=1),train['label'],test_size=0.3, random_state=0)
# X_train,X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=0)



In [11]:
#交叉验证的LGB
from sklearn.model_selection import KFold
# model = model.Booster(model_file='lgb.model')
params = {
    'task': 'train',
    'boosting_type': 'gbdt',  # 设置提升类型
    'objective': 'regression', # 目标函数
    'metric': {'binary_logloss'},  # 评估函数
    # 'num_leaves': 32,   # 叶子节点数
    # 'max_depth':5,   #最大数的深度
    'learning_rate': 0.2,  # 学习速率0.05 0.1
    # 'feature_fraction': 0.9, # 建树的特征选择比例
    # 'bagging_fraction': 0.9, # 建树的样本采样比例
    # 'bagging_freq': 5,  # k 意味着每 k 次迭代执行bagging
    #  'lambda': 5,
    'verbose': 1 # <0 显示致命的, =0 显示错误 (警告), >0 显示信息
}
n_splits = 5
oof_train = np.zeros((train.shape[0]))
oof_test = np.zeros((test.shape[0], n_splits))
train = train.drop(['query','title','query_id','query_title_id'],axis=1)

kfold = KFold(n_splits = n_splits, random_state=1337)
i = 0
print('Starting training ...')
for train_index, valid_index in kfold.split(train, train['label'].values):
    # print(train_index)
    X_tr = train.iloc[train_index, :]
    X_val = train.iloc[valid_index, :]
    
    y_tr = X_tr['label'].values
    X_tr = X_tr.drop(['label'], axis=1)
    
    y_val = X_val['label'].values
    X_val = X_val.drop(['label'], axis=1)
    
    print('\ny_tr distribution: {}'.format(Counter(y_tr)))
    
    d_train = lgb.Dataset(X_tr, label=y_tr) 
    d_valid = lgb.Dataset(X_val, label=y_val)
    watchlist = [d_train, d_valid]
    
    print('training LGB:')
    model = lgb.train(params,
                      train_set=d_train,
                      num_boost_round=50,
                      valid_sets=watchlist,
                      verbose_eval=1,
                      early_stopping_rounds=5)
    
    val_pred = model.predict(X_val, num_iteration=model.best_iteration)
    test_pred = model.predict(test.drop(['query','title','query_id','query_title_id'],axis=1), num_iteration=model.best_iteration)
    
    oof_train[valid_index] = val_pred
    oof_test[:, i] = test_pred
    i += 1
model.save_model('lgb.model')  # 用于存储训练出的模型

res = oof_test.mean(axis=1) 

Starting training ...

y_tr distribution: Counter({0: 5958083, 1: 2041917})
training LGB:
[1]	training's binary_logloss: 0.562131	valid_1's binary_logloss: 0.562674
Training until validation scores don't improve for 5 rounds.
[2]	training's binary_logloss: 0.558207	valid_1's binary_logloss: 0.558745
[3]	training's binary_logloss: 0.555457	valid_1's binary_logloss: 0.556009
[4]	training's binary_logloss: 0.553539	valid_1's binary_logloss: 0.554101
[5]	training's binary_logloss: 0.552184	valid_1's binary_logloss: 0.552761
[6]	training's binary_logloss: 0.551148	valid_1's binary_logloss: 0.551731
[7]	training's binary_logloss: 0.550382	valid_1's binary_logloss: 0.550955
[8]	training's binary_logloss: 0.549701	valid_1's binary_logloss: 0.550268
[9]	training's binary_logloss: 0.549222	valid_1's binary_logloss: 0.549803
[10]	training's binary_logloss: 0.548741	valid_1's binary_logloss: 0.549332
[11]	training's binary_logloss: 0.548374	valid_1's binary_logloss: 0.548976
[12]	training's binary

In [7]:

# 创建成lgb特征的数据集格式
lgb_train = lgb.Dataset(X_train.drop(['query_id','query_title_id','query','title'],axis=1), y_train)
#If this is Dataset for validation, training data should be used as reference.
lgb_eval = lgb.Dataset(X_test.drop(['query_id','query_title_id','query','title'],axis=1), y_test, reference=lgb_train)

#lgb_train = lgb.Dataset(X_train.drop(['query','title'],axis=1), label=y_train)
#lgb_eval = lgb.Dataset(X_test.drop(['query','title'],axis=1), label=y_test, reference=lgb_train)

# lgb_train = lgb.Dataset(X_train, y_train)
# lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
# 将参数写成字典下形式
params = {
    'task': 'train',
    'boosting_type': 'gbdt',  # 设置提升类型
    'objective': 'regression', # 目标函数
    'metric': {'binary_logloss'},  # 评估函数
    # 'num_leaves': 60,   # 叶子节点数
    # 'max_depth':5,   #最大数的深度
    'learning_rate': 0.2,  # 学习速率0.05 0.1
    # 'feature_fraction': 0.9, # 建树的特征选择比例
    # 'bagging_fraction': 0.9, # 建树的样本采样比例
    'bagging_freq': 5,  # k 意味着每 k 次迭代执行bagging
    #  'lambda': 10,
    'verbose': 1 # <0 显示致命的, =0 显示错误 (警告), >0 显示信息
}
watchlist = [lgb_train, lgb_eval]
# watchlist = [(lgb_train, 'train'), (lgb_eval, 'valid')]
print('Start training...')
# 训练 cv and train
gbm = lgb.train(params,lgb_train,num_boost_round=50,valid_sets = watchlist,early_stopping_rounds=5)

# print('Save model...')
# 保存模型到文件
#gbm.save_model('LightGBM.txt')

#print('Start predicting...')

# 预测数据集
#y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
#auc 0.544 


Start training...
[1]	training's binary_logloss: 0.562283	valid_1's binary_logloss: 0.562158
Training until validation scores don't improve for 5 rounds.
[2]	training's binary_logloss: 0.55836	valid_1's binary_logloss: 0.558237
[3]	training's binary_logloss: 0.555599	valid_1's binary_logloss: 0.555484
[4]	training's binary_logloss: 0.553701	valid_1's binary_logloss: 0.553584
[5]	training's binary_logloss: 0.552322	valid_1's binary_logloss: 0.552205
[6]	training's binary_logloss: 0.551287	valid_1's binary_logloss: 0.551185
[7]	training's binary_logloss: 0.550455	valid_1's binary_logloss: 0.550357
[8]	training's binary_logloss: 0.549845	valid_1's binary_logloss: 0.549752
[9]	training's binary_logloss: 0.549364	valid_1's binary_logloss: 0.549279
[10]	training's binary_logloss: 0.548962	valid_1's binary_logloss: 0.54888
[11]	training's binary_logloss: 0.548605	valid_1's binary_logloss: 0.548527
[12]	training's binary_logloss: 0.548008	valid_1's binary_logloss: 0.547927
[13]	training's bina

In [10]:
def q_auc(df):
    invalid_count = 0
    query_ids = df['query_id'].unique()
    aucs = []
    for query_id in query_ids:
        current = df[df['query_id'] == query_id]
        if sum(current['label']) not in [0, len(current['label'])]:
            auc = roc_auc_score(current['label'], current['prediction'])  # 这里计算qauc
        else:
            auc = 0.5
            invalid_count += 1
        aucs.append(auc)
    print(f'q_auc invalid_count = {invalid_count}')
    return np.mean(aucs)
x = train[:100000]
# x = train[:10000]
# x.drop(['query_id','query_title_id','title','query','label'],axis=1)
pred = gbm.predict(x.drop(['query_id','query_title_id','title','query','label'],axis=1), num_iteration=gbm.best_iteration)
x['prediction'] = pred
#qauc 0.537-> 0.536-> 0.538-> 0.544-> 0.545-> 0.547->0.564(特征工程)->0.561（lambda 10 正则）
q_auc(x) 

q_auc invalid_count = 363


0.5351825462797781

In [8]:
import seaborn as sns
# feature_imp = pd.DataFrame(sorted(zip(clf.feature_importances_,X.columns)), columns=['Value','Feature'])
plt.figure(figsize=(20, 10))
importance = pd.DataFrame({'column': gbm.feature_name(),'importance': gbm.feature_importance(),}).sort_values(by='importance',ascending=False)
sns.barplot(x="importance", y="column", data=importance)
plt.title('LightGBM Features Importance')
plt.tight_layout()
plt.show()

# importance.plot()

In [24]:
importance


In [ ]:
# test['prediction'] = gbm.predict(test.drop(['query_id','query_title_id','title','query'],axis=1))
test['prediction'] = res
sub = test[['query_id','query_title_id','prediction']]
sub.to_csv('lgb_1.csv',index=False,header=False)


In [23]:
!head -n 10 lgb_1.csv

In [ ]:
# 下载提交工具至当前目录，仅需执行一次（ XXXXXXXXX 为团队 Token）
!wget -nv -O kesci_submit https://www.heywhale.com/kesci_submit&&chmod +x kesci_submit
# 提交文件 my_submission.csv 进行评审；
!./kesci_submit -token 2ebe9c16d35d9efc -file lgb_1.csv